In [1]:
import pandas as pd
import numpy as np
df_train = pd.read_csv('./train.csv', header=0)
df_test = pd.read_csv('./test.csv', header=0)

In [2]:
df_train['Gender']=4
df_train['Gender'] = df_train['Sex'].map( {'female':0, 'male':1}).astype(int)

median_ages = np.zeros((2,3))
for i in range(0,2):
    for j in range(0,3):
        median_ages[i,j] = df_train[(df_train['Gender'] == i) & 
                              (df_train['Pclass'] == j+1)]['Age'].dropna().median()
df_train['AgeFill'] = df_train['Age']
df_train['AgeIsNull'] = pd.isnull(df_train.Age).astype(int)
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch']
df_train['Age*Class'] = df_train.AgeFill * df_train.Pclass
df_train = df_train.drop(['Name','Sex','Ticket','Cabin','Embarked', 'Age'], axis=1)
df_train = df_train.dropna()

In [3]:
df_test['Gender']=4
df_test['Gender'] = df_test['Sex'].map( {'female':0, 'male':1}).astype(int)

median_ages = np.zeros((2,3))
for i in range(0,2):
    for j in range(0,3):
        median_ages[i,j] = df_test[(df_test['Gender'] == i) & 
                              (df_test['Pclass'] == j+1)]['Age'].dropna().median()
df_test['AgeFill'] = df_test['Age']
df_test['AgeIsNull'] = pd.isnull(df_test.Age).astype(int)
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch']
df_test['Age*Class'] = df_test.AgeFill * df_test.Pclass
df_test['Survivid']=0
df_test = df_test.drop(['Name','Sex','Ticket','Cabin','Embarked', 'Age'], axis=1)
df_test = df_test.dropna()

In [10]:
train_data = df_train.values
test_data = df_test.values
# Import the random forest package
from sklearn.ensemble import RandomForestClassifier 

# Create the random forest object which will include all the parameters
# for the fit
forest = RandomForestClassifier(n_estimators = 100)

# Fit the training data to the Survived labels and create the decision trees
# train_data [0::, 1::]
# 0번째 row부터 다 가져올 것
# 1번째 colum부터 다 가져올 것
# train_data[0::,0]
forest = forest.fit(train_data,train_data[0::,1])

# Take the same decision trees and run it on the test data
output = forest.predict(test_data)

In [9]:
output.size

331